# Test diffusion models

## 1. Workspace setup

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [ ]:
import yaml
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchmetrics
import pandas as pd
from diffusers import UNet2DModel, DDPMScheduler
from spinediffusion.models.diffusion_models import UnconditionalDiffusionModel
from pathlib import Path
from tensorflow.python.summary.summary_iterator import summary_iterator

## 2. Load data

### 2.1. CSV logs

This data has been previously transformed to csv format at the end of training by a pytorch callback and saved to disk. For more information refer to the source code of the `GenerateCSVLog` within the `callbacks.py` file.

In [ ]:
log_paths = [
    Path(
        f"P:\\Projects\\LMB_4Dspine\\Iship_Pau_Altur_Pastor\\4_training_logs\\logs\\depthmap\\version_{i}\events.csv"
    )
    for i in range(6, 11)
]

df_tf = pd.DataFrame(columns=["run_name", "time", "tag", "value"])

for path in log_paths:
    run_name = path.parent.stem

    df_run = pd.read_csv(path)
    df_run["run_name"] = run_name

    df_tf = pd.concat([df_tf, df_run])

df_tf = df_tf.sort_values(by=["run_name", "tag", "time"])
df_tf["step"] = df_tf.groupby(["run_name", "tag"]).cumcount()

df_tf

In [ ]:
df_tf.sort_values(["run_name", "tag", "time"])

### 2.2. Load config files

In [ ]:
configs = {}

for path in log_paths:
    run_name = path.parent.stem

    with open(path.parent / "config.yaml", "r") as f:
        config = yaml.safe_load(f)

    configs[run_name] = config

### 2.3. Combine them

In [ ]:
for run in configs:
    df_tf[df_tf["run_name"] == run]["lr"] = configs[run]["optimizer"]["init_args"]["lr"]

    max_epochs = configs[run]["trainer"]["max_epochs"]
    max_steps = df_tf[df_tf["run_name"] == run]["step"].max()
    df_tf[df_tf["run_name"] == run]["epoch"] = (
        df_tf[df_tf["run_name"] == run]["step"] // max_steps
    ) * max_epochs

df_tf

## 3. Plot training curves

### Per run

In [ ]:
keys = [
    "train_loss_step",
    "MSELoss",
    "PSNR",
    "SSIM",
    "val_loss_step",
    "val_loss_epoch",
    "train_loss_epoch",
]

for run in df_tf.run_name.unique():
    df_run = df_tf[df_tf.run_name == run]
    lr = configs[run]["optimizer"]["init_args"]["lr"]

    for key in keys:
        df_run_key = df_run[df_run.tag == key]
        plt.plot(df_run_key.value)
        plt.title(f"{run} - {lr}")
        plt.xlabel("time")
        plt.ylabel(key)
        plt.grid()
        plt.show()

### Compare runs

In [ ]:
for key in keys:
    for run in df_tf.run_name.unique():
        df_run_key = df_tf[(df_tf.run_name == run) & (df_tf.tag == key)]
        plt.plot(df_run_key.value, label=configs[run]["optimizer"]["init_args"]["lr"])

    plt.title(f"{key} vs step")
    plt.xlabel("step")
    plt.ylabel(key)
    plt.legend()
    plt.grid()
    plt.show()

## 3. Visualize tensorflow logs

In [ ]:
df_tf = pd.DataFrame()

for e in summary_iterator(
    str(log_path / "events.out.tfevents.1719496170.Portatil_Pau.95140.0")
):
    if len(e.summary.value) == 0:
        continue
    index = e.wall_time
    tag = e.summary.value[0].tag
    value = e.summary.value[0].simple_value
    df_tf.loc[index, tag] = value

In [ ]:
df_tf

In [ ]:
import numpy as np

np.diff(df_tf.index.values())

## 4. Test inference